## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-13-16-46-00 +0000,wsj,U.S. Troops in Syria Wounded in Ambush,https://www.wsj.com/world/middle-east/u-s-troo...
1,2025-12-13-16-42-55 +0000,nypost,Iran’s gas prices soar for the first time sinc...,https://nypost.com/2025/12/13/world-news/irans...
2,2025-12-13-16-41-20 +0000,nyt,"Israel Says It Killed Senior Hamas Commander, ...",https://www.nytimes.com/2025/12/13/world/middl...
3,2025-12-13-16-34-44 +0000,cbc,"Belarus frees Nobel winner, top opposition fig...",https://www.cbc.ca/news/world/belarus-frees-pr...
4,2025-12-13-16-33-10 +0000,bbc,Belarus frees 123 prisoners as US lifts sanctions,https://www.bbc.com/news/articles/c4gpwlkklyxo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2406/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
30,trump,35
124,new,11
164,was,11
107,year,11
194,russia,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
203,2025-12-12-21-34-00 +0000,wsj,President Trump said he was leaning toward cho...,https://www.wsj.com/economy/central-banking/tr...,111
164,2025-12-12-23-35-12 +0000,nypost,Unredacted Trump photos from Epstein files dum...,https://nypost.com/2025/12/12/us-news/unredact...,76
177,2025-12-12-22-48-29 +0000,nyt,How Ukraine Has Responded to Trump’s Peace Pla...,https://www.nytimes.com/2025/12/12/world/europ...,72
149,2025-12-13-00-35-21 +0000,bbc,"As Trump targets Venezuela, its allies Russia ...",https://www.bbc.com/news/articles/c075mr3190mo...,71
237,2025-12-12-19-33-00 +0000,wsj,New Epstein Photos Released by Democrats Show ...,https://www.wsj.com/politics/democrats-release...,68


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
203,111,2025-12-12-21-34-00 +0000,wsj,President Trump said he was leaning toward cho...,https://www.wsj.com/economy/central-banking/tr...
27,43,2025-12-13-15-03-04 +0000,nyt,Russia Bombs Ukraine Port of Odesa Amid Peace ...,https://www.nytimes.com/2025/12/13/world/europ...
50,41,2025-12-13-12-55-06 +0000,nypost,"Public urination, drug use, other NYC quality-...",https://nypost.com/2025/12/13/us-news/public-u...
28,33,2025-12-13-14-48-43 +0000,cbc,"Organizer arrested after fans rip up seats, th...",https://www.cbc.ca/news/world/soccer-fans-mess...
237,33,2025-12-12-19-33-00 +0000,wsj,New Epstein Photos Released by Democrats Show ...,https://www.wsj.com/politics/democrats-release...
230,32,2025-12-12-20-01-34 +0000,latimes,Leaders of secretive California religious grou...,https://www.latimes.com/california/story/2025-...
217,32,2025-12-12-20-35-30 +0000,nyt,Iran Arrests Nobel Laureate Narges Mohammadi A...,https://www.nytimes.com/2025/12/12/world/middl...
24,29,2025-12-13-15-49-43 +0000,nypost,Disgraced ex-University of Michigan football c...,https://nypost.com/2025/12/13/us-news/universi...
145,29,2025-12-13-01-09-53 +0000,nyt,House G.O.P. Releases Health Plan That Would A...,https://www.nytimes.com/2025/12/12/us/politics...
25,29,2025-12-13-15-26-20 +0000,nypost,Israel kills top Hamas commander Raad Saad wit...,https://nypost.com/2025/12/13/world-news/israe...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
